In [1]:
import os
import sys

from typing import List

from alibabacloud_devops20210625.client import Client as devops20210625Client
from alibabacloud_tea_openapi import models as open_api_models
from alibabacloud_tea_util import models as util_models
from alibabacloud_tea_util.client import Client as UtilClient
from alibabacloud_devops20210625 import models as devops_20210625_models

ALIBABA_CLOUD_ACCESS_KEY_ID = os.environ.get(
    "ALIBABA_CLOUD_ACCESS_KEY_ID", "请配置：ALIBABA_CLOUD_ACCESS_KEY_ID"
)  # 也可以直接把ACCESS_KEY_ID放在这里

ALIBABA_CLOUD_ACCESS_KEY_SECRET = os.environ.get(
    "ALIBABA_CLOUD_ACCESS_KEY_SECRET", "请配置：ALIBABA_CLOUD_ACCESS_KEY_SECRET"
)  # 也可以直接把ACCESS_KEY_SECRET放在这里

# print(ALIBABA_CLOUD_ACCESS_KEY_ID, ALIBABA_CLOUD_ACCESS_KEY_SECRET)

# print(os.environ['ALIBABA_CLOUD_ACCESS_KEY_ID'], os.environ['ALIBABA_CLOUD_ACCESS_KEY_SECRET'])
config = open_api_models.Config(
            access_key_id=ALIBABA_CLOUD_ACCESS_KEY_ID,
            access_key_secret=ALIBABA_CLOUD_ACCESS_KEY_SECRET
        )
# Endpoint 请参考 https://api.aliyun.com/product/devops
config.endpoint = f'devops.cn-hangzhou.aliyuncs.com'
client = devops20210625Client(config)

In [2]:
# 项目列表

import json
import pandas as pd

runtime = util_models.RuntimeOptions()
headers = {}
organization_id = "6189f099041d450d2c253abc"
project_id = "a41d2acc436b9755f11b79d3d4"

# print(client.list_organizations(request=devops_20210625_models.ListOrganizationsRequest()).body)

req = devops_20210625_models.ListProjectWorkitemTypesRequest(
    space_type="Project", category="req"
)
res = client.list_project_workitem_types_with_options(
    organization_id=organization_id,
    project_id=project_id,
    headers=headers,
    runtime=runtime,
    request=req,
)
# print(res.body)

req_id = "1a787d02dbffcf9bbe93e73631"

req = devops_20210625_models.ListWorkItemAllFieldsRequest(
    space_type="Project", space_identifier=project_id, workitem_type_identifier=req_id
)
res = client.list_work_item_all_fields_with_options(
    organization_id=organization_id, headers=headers, runtime=runtime, request=req
)

# print(res.body)
# all_fields=json.loads(res.body)

fields_df = pd.DataFrame(res.body.to_map()["fields"])
fields_df.describe()

start_date = "2024-04-01"
end_date = "2024-06-30"

work_item_condition = {
    "conditionGroups": [
        [
            {
                "fieldIdentifier": "status",
                "operator": "CONTAINS",
                "value": ["100014"],
                "toValue": None,
                "className": "status",
                "format": "list",
            },
            {
                "fieldIdentifier": "finishTime",
                "operator": "BETWEEN",
                "value": [f"{start_date} 00:00:00"],
                "toValue": f"{end_date} 23:59:59",
                "className": "date",
                "format": "input",
            },
            {
                "fieldIdentifier": "tag",
                "operator": "CONTAINS",
                "value": ["02c574f4854f04fde8339ba2b1", "a6b2221931325308e07f88220f"],
                "toValue": None,
                "className": "tag",
                "format": "multiList",
            },
        ]
    ]
}
work_item_condition = json.dumps(work_item_condition)
# '{"conditionGroups":[[{"fieldIdentifier":"status","operator":"CONTAINS","value":["63798bd5f6855ea51abcd1b0d6"],"toValue":null,"className":"status","format":"list"},{"fieldIdentifier":"workitemType","operator":"CONTAINS","value":["1a787d02dbffcf9bbe93e73631"],"toValue":null,"className":"workitemType","format":"list"}]]}'
req = devops_20210625_models.ListWorkitemsRequest(
    space_type="Project",
    category="Req",
    conditions=work_item_condition,
    max_results=200,
    search_type="LIST",
    space_identifier=project_id,
)
res = client.list_workitems(
    organization_id=organization_id,
    #    headers=headers,
    #    runtime=runtime,
    request=req,
)

items_array = res.body.to_map()["workitems"]


# 将这段代码转成小方法，参数：workitem_id,organization_id
def get_work_item_info(workitem_id, organization_id):
    work_item = client.get_work_item_info(
        workitem_id=workitem_id, organization_id=organization_id
    )
    return work_item.body.to_map()["workitem"]


# 循环items_array，使用每个workitem的id，调用get_work_item_info方法，获取workitem的详细信息，并将结果保存到一个列表中
work_item_info_list = []
for item in items_array:
    work_item_info_list.append(get_work_item_info(item["identifier"], organization_id))

In [3]:
fields_dict = {}
for index, row in fields_df.iterrows():
    row_data = row.to_dict()  # Convert the row to a dictionary
    fields_dict[row_data['identifier']] = row_data['name']

fields_dict

{'subject': '标题',
 'description': '描述',
 'parent': '父项ID',
 'parentWhetherExist': '父项是否存在',
 'parentSubject': '父项标题',
 'workitemType': '工作项类型',
 'status': '状态',
 'assignedTo': '负责人',
 'e4c05dae33ce16b6e3ed4f65ea': '测试负责人',
 'priority': '优先级',
 'space': '归属项目',
 'sprint': '迭代',
 'ak.issue.member': '参与者',
 'workitem.tracker': '抄送',
 'tag': '标签',
 '79': '计划开始时间',
 '80': '计划完成时间',
 'relatedSpace': '共享项目',
 'a6f2cf7e6c69e5006ac4cd5146': '变更后完成时间',
 'version': '版本',
 '2000c2aad751b7871d0895d7d4': '需求评审不通过原因',
 'edd9f7d98179e7f47fae41b135': 'UI评审不通过原因',
 '51ec6c83a2b9c10d5b437ab616': '提测打回原因',
 '8159aca815ebf0b25a9e4d0d67': '提测打回责任人',
 'a08493245a9cf9fe250677233e': '产品验收不通过原因',
 '2c322e0439231d088adafdf7c2': 'UI验收不通过原因',
 '1cf9f1e546618af480f9cec18c': '需求延期类型',
 'b37f146595fd4734d69675690c': '需求延期原因',
 '16b58fbf0eb1556eaacffcd6d5': '延期责任人',
 'fba0a046ffed2f9c5224db5493': '归属业务'}

In [4]:
fields_dict

# 获取工作项的活动记录
def get_work_item_activities(workitem):
    activities=client.get_work_item_activity(organization_id=organization_id, workitem_id=workitem['identifier']).body
    activities_dict={}
    for act in activities.to_map()['activities']:
        if act['eventType'] == 'workitem.transitioned':
            activity_name = act['newValue'][0]['displayValue']
            if activity_name in activities_dict:
                new_value=pd.to_datetime(act['eventTime'], unit='ms').strftime('%Y-%m-%d %H:%M:%S')
                old_value=activities_dict[activity_name]
                if new_value > old_value:
                    activities_dict[activity_name]=new_value
            else:
                activities_dict[activity_name]=pd.to_datetime(act['eventTime'], unit='ms').strftime('%Y-%m-%d %H:%M:%S')
    print(activities_dict)
    workitem['activities_dict']=activities_dict
    return workitem

# 遍历所有项目，获取每个项目的活动记录
for work_item in work_item_info_list:
    workitems=get_work_item_activities(work_item)

for item in work_item_info_list:
    custom_fields_dict = {}
    for custom_field in item['customFields']:
        fieldIdentifier=custom_field['fieldIdentifier']
        if fieldIdentifier in fields_dict:
            custom_fields_dict[fields_dict[fieldIdentifier]] = custom_field['value']
        else:
            custom_fields_dict[fieldIdentifier] = custom_field['value']
    print(custom_fields_dict)
    get_work_item_activities(workitem=item)
    item['custom_fields_dict'] = custom_fields_dict

{'已完成': '2024-06-06 02:03:14', '待发布': '2024-06-06 02:03:11', '待测试': '2024-06-06 02:03:01', '开发中': '2024-06-05 10:31:26', '需求&UI评审完成': '2024-06-05 10:31:17', '待需求评审': '2024-06-05 07:40:28', '需求设计中': '2024-06-05 07:40:24'}
{'已完成': '2024-06-07 01:30:43', '待产品&UI验收': '2024-06-06 10:08:27', '测试中': '2024-06-06 10:07:43', '待测试': '2024-06-06 10:07:34', '开发中': '2024-06-05 10:31:15', '技术评审完成': '2024-06-04 08:50:32', '技术方案设计中': '2024-06-04 07:14:24', '需求&UI评审完成': '2024-06-04 07:14:21', '待需求评审': '2024-06-04 07:13:07', '待需求预审': '2024-06-04 07:12:47', '需求设计中': '2024-06-04 07:12:45'}
{'已完成': '2024-06-07 02:07:06', '待发布': '2024-06-07 02:06:58', '待测试': '2024-06-07 02:06:45', '开发中': '2024-06-05 02:08:44', '技术评审完成': '2024-06-05 02:08:39', '技术方案设计中': '2024-06-03 11:02:45', '需求&UI评审完成': '2024-06-03 11:02:42'}
{'已完成': '2024-06-07 01:23:14', '待测试': '2024-06-05 10:32:05', '开发中': '2024-05-30 01:54:19', '需求&UI评审完成': '2024-05-30 01:53:59'}
{'已完成': '2024-06-05 10:49:19', '待测试': '2024-06-05 10:49:14', '开发中': '2024

In [5]:
items_df=pd.DataFrame(work_item_info_list)
items_df['gmtModified'] = pd.to_datetime(items_df['gmtModified'], unit='ms')
items_df['finishTime'] = pd.to_datetime(items_df['finishTime'], unit='ms')
items_df['gmtCreate'] = pd.to_datetime(items_df['gmtCreate'], unit='ms')
items_df['updateStatusAt'] = pd.to_datetime(items_df['updateStatusAt'], unit='ms')
# Convert the specific key value to datetime
items_df['研发开始时间'] = items_df['custom_fields_dict'].apply(lambda x: pd.to_datetime(x.get('计划开始时间')))
items_df['计划发布时间'] = items_df['custom_fields_dict'].apply(lambda x: pd.to_datetime(x.get('计划完成时间')))
# sumActualLaborHour
items_df['需求总工时'] = items_df['custom_fields_dict'].apply(lambda x: pd.to_numeric(x.get('sumActualLaborHour')))

# Calculate the difference in seconds between two datetime columns
items_df['研发人日'] = (items_df['计划发布时间'] - items_df['研发开始时间']).dt.total_seconds() / (24 * 3600)

# Round the result to one decimal place
items_df['研发人日'] = items_df['研发人日'].round(1)
# Create a new column '是否延期' with boolean values
items_df['是否延期'] = items_df['finishTime'].dt.date > items_df['计划发布时间'].dt.date

# Convert boolean values to 1 for True and 0 for False
items_df['是否延期'] = items_df['是否延期'].astype(int)
items_df.describe()

,需求总工时,研发人日,是否延期
count,39.000000,39.000000,39.000000
mean,33.935897,8.692308,0.153846
std,37.503365,10.438173,0.365518
min,0.500000,0.000000,0.000000
25%,9.000000,2.500000,0.000000
50%,24.000000,5.000000,0.000000
75%,43.000000,11.000000,0.000000
max,171.000000,44.000000,1.000000


In [6]:
import datetime

yyyymmdd=datetime.datetime.now().strftime('%Y-%m-%d')
items_df.to_csv(f'./data/saas云效-{start_date}_{end_date}_at_{yyyymmdd}.csv', index=False)

In [7]:
# Set the option to display full column width
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)  # Set to None to display all rows
pd.set_option('display.max_columns', None)  # Set to None to display all columns
pd.set_option('display.width', None)  # Set width to None for automatic wrapping

all_timing_fields=['已完成','待产品&UI验收','待测试','开发中','需求&UI评审完成','待需求评审','测试中','技术评审完成','技术方案设计中','需求设计中','待发布']
items_to_analytics_df=items_df[['subject','研发人日','是否延期','需求总工时','activities_dict','gmtCreate','计划发布时间']]
for timing_field in all_timing_fields:
    items_to_analytics_df[timing_field]=items_to_analytics_df['activities_dict'].apply(lambda x: pd.to_datetime(x.get(timing_field)))

items_to_analytics_df['需求分析时长D'] = (items_to_analytics_df['需求&UI评审完成'] - items_to_analytics_df['需求设计中']).dt.total_seconds() / (24 * 3600)
items_to_analytics_df['需求纯开发时长D'] = (items_to_analytics_df['待测试'] - items_to_analytics_df['开发中']).dt.total_seconds() / (24 * 3600)
items_to_analytics_df['需求纯测试时长D'] = (items_to_analytics_df['待产品&UI验收'] - items_to_analytics_df['待测试']).dt.total_seconds() / (24 * 3600)
items_to_analytics_df['需求纯技术设计时长D'] = (items_to_analytics_df['开发中'] - items_to_analytics_df['需求&UI评审完成']).dt.total_seconds() / (24 * 3600)
# Calculate the difference in seconds between two datetime columns
items_to_analytics_df['研发人日'] = (items_to_analytics_df['已完成'] - items_to_analytics_df['需求&UI评审完成']).dt.total_seconds() / (24 * 3600)
items_to_analytics_df['已完成-需求&UI评审完成'] = (items_to_analytics_df['已完成'] - items_to_analytics_df['需求&UI评审完成']).dt.days
items_to_analytics_df.head(1)

/var/folders/cl/v_4j9fbj5nn_jj6q9d3r5ggc0000gn/T/ipykernel_83375/2564869647.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  items_to_analytics_df[timing_field]=items_to_analytics_df['activities_dict'].apply(lambda x: pd.to_datetime(x.get(timing_field)))
/var/folders/cl/v_4j9fbj5nn_jj6q9d3r5ggc0000gn/T/ipykernel_83375/2564869647.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  items_to_analytics_df['需求分析时长D'] = (items_to_analytics_df['需求&UI评审完成'] - items_to_analytics_df['需求设计中']).dt.total_seconds() 

,subject,研发人日,是否延期,需求总工时,activities_dict,gmtCreate,计划发布时间,已完成,待产品&UI验收,待测试,开发中,需求&UI评审完成,待需求评审,测试中,技术评审完成,技术方案设计中,需求设计中,待发布,需求分析时长D,需求纯开发时长D,需求纯测试时长D,需求纯技术设计时长D,已完成-需求&UI评审完成
0,【CRM】任务标签名可配置,0.647188,1,2.0,"{'已完成': '2024-06-06 02:03:14', '待发布': '2024-06-06 02:03:11', '待测试': '2024-06-06 02:03:01', '开发中': '2024-06-05 10:31:26', '需求&UI评审完成': '2024-06-05 10:31:17', '待需求评审': '2024-06-05 07:40:28', '需求设计中': '2024-06-05 07:40:24'}",2024-06-05 07:40:17,2024-06-05,2024-06-06 02:03:14,NaT,2024-06-06 02:03:01,2024-06-05 10:31:26,2024-06-05 10:31:17,2024-06-05 07:40:28,NaT,NaT,NaT,2024-06-05 07:40:24,2024-06-06 02:03:11,0.118669,0.646933,NaN,0.000104,0


## 研发人日小于等于10天需求百分比%

In [8]:
import pandas as pd
import pandasql

column_name = 'activities_dict'
if column_name in items_to_analytics_df.columns:
    items_to_analytics_df.drop(columns=column_name, inplace=True)
# items_to_analytics_df.drop("activities_dict")
stat=pandasql.sqldf("""
                    select 
                    count(1) as `需求总数`,
                    count(case when `研发人日` >= 11 then 1 end) as `研发人日大于10天需求数`,
                    round(count(case when `研发人日` < 11 then 1 end)*100.00/count(1),2) as `研发人日小于等于10天需求百分比%`,
                    round(sum(`研发人日`)/count(1),2) as `avg研发人日D`,
                    round(sum(`需求总工时`)/count(1),2) as `avg研发工时H`,
                    count(case when `是否延期` > 0 then 1 end) as `延期需求数`,
                    count(case when `需求总工时` > 100 then 1 end) as `100h以上的需求数`
                    from items_to_analytics_df
                    """)

display(stat)

/Users/tangpeng/Documents/github/.venv/lib/python3.10/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,需求总数,研发人日大于10天需求数,研发人日小于等于10天需求百分比%,avg研发人日D,avg研发工时H,延期需求数,100h以上的需求数
0,39,10,74.36,8.74,33.94,6,2


In [9]:
stat=pandasql.sqldf("""
                    select 
                    round(avg(`需求分析时长D`),2) as `avg需求分析时长D`,
                    round(avg(`需求纯开发时长D`),2) as `avg需求纯开发时长D`,
                    round(avg(`需求纯测试时长D`),2) as `avg需求纯测试时长D`,
                    round(avg(`需求纯技术设计时长D`),2) as `需求纯技术设计时长D`,
                    count(case when `需求纯开发时长D` >= 7 then 1 end) as `需求纯开发时长D>=7需求数`,
                    count(case when `需求纯测试时长D` >= 3 then 1 end) as `需求纯测试时长D>=3需求数`,
                    count(case when `需求纯技术设计时长D` >= 2 then 1 end) as `需求纯技术设计D>=2需求数`,
                    count(1) as `总需求数`
                    from items_to_analytics_df
                    """)
display(stat)

,avg需求分析时长D,avg需求纯开发时长D,avg需求纯测试时长D,需求纯技术设计时长D,需求纯开发时长D>=7需求数,需求纯测试时长D>=3需求数,需求纯技术设计D>=2需求数,总需求数
0,4.64,3.86,1.94,3.24,9,5,12,39


## 研发人日>=10天的项目列表

In [10]:
from IPython.display import display

stat=pandasql.sqldf("""
                    select 
                    subject
                    ,round(`研发人日`,1) as `研发人日D`
                    ,round(`需求纯开发时长D`,1) as `需求纯开发时长D`
                    ,round(`需求纯测试时长D`,1) as `需求纯测试时长D`
                    ,date(`已完成`) as `完成日期`
                    ,date(`需求&UI评审完成`) as `需求评审完成日期`
                    ,date(`开发中`) as `首次进入开发中时间`
                    ,case when date(`需求&UI评审完成`) <= date('2024-05-01') then '五一前评审' else '否' end as `是否经历假期`
                    ,`已完成-需求&UI评审完成` as `研发时长D`
                    ,`需求总工时`,
                    round(`需求总工时`/`已完成-需求&UI评审完成`,1) as `avg工时per研发人日`
                    from items_to_analytics_df
                    where `研发人日` >=11
                    and `需求总工时`>0
                    order by `完成日期` desc
                    """)
# Set the option to display full column width
pd.set_option('display.max_colwidth', None)
display(stat)

,subject,研发人日D,需求纯开发时长D,需求纯测试时长D,完成日期,需求评审完成日期,首次进入开发中时间,是否经历假期,研发时长D,需求总工时,avg工时per研发人日
0,SaaS支持快递链路,13.4,8.0,3.0,2024-06-03,2024-05-21,2024-05-23,否,13,83.5,6.4
1,对接三方仓(绝配)打通订单流,41.3,14.0,NaN,2024-05-28,2024-04-17,2024-05-06,五一前评审,41,32.5,0.8
2,【财务】销项发票切换为数电票,31.3,10.4,7.0,2024-05-27,2024-04-26,2024-05-10,五一前评审,31,80.0,2.6
3,商品管理-商品标签能力,23.0,12.9,NaN,2024-05-15,2024-04-22,2024-04-28,五一前评审,23,135.0,5.9
4,【CRM】5月鲜果/PT奶油任务产品化,14.0,10.0,NaN,2024-05-10,2024-04-26,2024-04-30,五一前评审,14,52.5,3.8
5,【SaaS】帆台官网For ICP：增加付费资讯栏目,13.0,8.7,NaN,2024-05-07,2024-04-24,2024-04-28,五一前评审,13,24.0,1.8
6,全品类接入,16.1,7.0,6.9,2024-04-23,2024-04-07,2024-04-09,五一前评审,16,171.0,10.7
7,定价分析,38.6,0.0,NaN,2024-04-16,2024-03-08,2024-04-16,五一前评审,38,88.0,2.3
8,‍​⁢﻿‍‬‬‬‍﻿‍‬‬‍⁢⁣‍​﻿⁢‌⁢‬​⁤‌‍⁢​‍⁣﻿‍‬⁣⁢⁣⁣⁡⁤‌‍⁤⁢‍⁤⁢‌​​帆台-售后支持上传视频素材,15.2,9.7,1.2,2024-04-09,2024-03-25,2024-03-29,五一前评审,15,50.0,3.3
9,需求-帆台报价模式升级,19.3,13.3,5.9,2024-04-07,2024-03-19,2024-03-19,五一前评审,19,72.5,3.8


## 延期的项目

In [11]:
stat=pandasql.sqldf("""
                    select 
                    subject
                    ,round(`研发人日`,1) as `研发人日D`
                    ,date(`已完成`) as `完成日期`
                    ,date(`计划发布时间`) as `计划发布时间`
                    ,date(`需求&UI评审完成`) as `需求评审完成日期`
                    ,date(`开发中`) as `首次进入开发中时间`
                    ,case when date(`需求&UI评审完成`) <= date('2023-10-07') then '十一前评审' else '否' end as `是否经历假期`
                    ,`已完成-需求&UI评审完成` as `研发时长D`
                    ,`需求总工时`,
                    round(`需求总工时`/`已完成-需求&UI评审完成`,1) as `avg工时per研发人日`
                    from items_to_analytics_df
                    where `是否延期` > 0
                    order by `完成日期` desc
                    """)
display(stat)

,subject,研发人日D,完成日期,计划发布时间,需求评审完成日期,首次进入开发中时间,是否经历假期,研发时长D,需求总工时,avg工时per研发人日
0,【商城】控价品下单链路隐藏优惠信息,2.8,2024-06-07,2024-06-06,2024-06-04,2024-06-05,否,2,21.0,10.5
1,【619】鲜沐年度账单,8.0,2024-06-07,2024-06-04,2024-05-30,2024-05-30,否,7,32.5,4.6
2,【CRM】任务标签名可配置,0.6,2024-06-06,2024-06-05,2024-06-05,2024-06-05,否,0,2.0,NaN
3,【CRM】企微销售-客户关系自动迁移,4.2,2024-05-24,2024-05-21,2024-05-20,2024-05-20,否,4,31.0,7.8
4,【CRM】5月鲜果/PT奶油任务产品化,14.0,2024-05-10,2024-04-30,2024-04-26,2024-04-30,否,14,52.5,3.8
5,需求-帆台报价模式升级,19.3,2024-04-07,2024-03-27,2024-03-19,2024-03-19,否,19,72.5,3.8


In [12]:
df=items_to_analytics_df[['subject','已完成']].sort_values('已完成', ascending=False)
df.head(10)

,subject,已完成
2,商城&CRM开票入口切换,2024-06-07 02:07:06
1,【商城】控价品下单链路隐藏优惠信息,2024-06-07 01:30:43
3,【619】鲜沐年度账单,2024-06-07 01:23:14
0,【CRM】任务标签名可配置,2024-06-06 02:03:14
4,【CRM】企微会话存档数据调取,2024-06-05 10:49:19
5,【营销】满返支持下单返券,2024-06-05 10:31:04
12,【商城】鲜果新增单价展示,2024-06-05 08:55:35
17,SaaS支持快递链路,2024-06-03 11:03:13
6,存量发票开具重试,2024-06-03 02:12:20
25,对接三方仓(绝配)打通订单流,2024-05-28 09:58:01
